In [ ]:
import xarray as xr
from data_loading.load_ens import *

ds = xr.open_mfdataset('../data/mars_v05e_*.grib', engine='cfgrib', concat_dim='time', combine='nested', parallel=True, chunks={'time': 1})
ds = calculate_wind_speed(ds)
ds = calculate_temperature_in_C(ds)
ds = get_diff_values(ds)

# use Dask to open multiple E files for distributed computing (1 chunk per file)
# ds, dsD = load_multiple_ens_data_ED('../data')
ds

In [ ]:
def calculate_wind_power(speed, C=0.612):
    return C * speed**3  

In [ ]:
# calculate wind power
ds = ds.assign(p10=calculate_wind_power(ds['w10']),
               p100=calculate_wind_power(ds['w100']))
ds

In [ ]:
from dask.diagnostics import ProgressBar
ProgressBar().register()

# plot average wind power over space
ds.p10.mean(dim=['number','time', 'step']).plot()

In [ ]:
# reduce the dataset to time series of spatial and ensemble means
from preprocessing.reducers import *
time_series = calculate_climatological_spatial_mean(ds)
time_series = time_series.groupby('valid_time').mean()
time_series = time_series.to_dataframe()
time_series.plot()

In [ ]:
ds['p10'].groupby('valid_time').mean(['number'])

In [ ]:
# calculate average over ensembles and files
cross_date_ds = cross_date_mean(ds)
cross_date_ds

In [ ]:
import plotly.express as px
px.imshow(cross_date_ds.w10, aspect='auto', animation_frame='valid_time')